<a href="https://colab.research.google.com/github/vlozg/speech_hmm/blob/main/Test_DiagHMM_005.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LƯU Ý: NOTEBOOK NÀY CHỈ DÙNG ĐỂ SHOW KẾT QUẢ CHẠY, KHÔNG NÊN CHẠY LẠI NOTEBOOK NÀY 
(VÌ TRONG NÀY CÓ CODE LƯU LẠI PRETRAINED MODEL LÊN DRIVE SẼ BỊ XÓA)

# Speech to text with HMM

- **Bài toán**: Chuyển giọng nói thành văn bản
    - **Input**: Đoạn ghi âm chứa nội dung là các số từ 0 đến 9
    - **Output**: Phân lớp của đoạn ghi âm

# Các biến thiết lập cho thử nghiệm

In [ ]:
n_mfcc_ceptrum = 12
n_delta_features = 1
fsdd_split = 0.6
wolfram_split = 0.6
experiment_id = '005'

In [ ]:
n_mfcc_features = n_mfcc_ceptrum * (1+n_delta_features)
n_mfcc_features

24

# Import và cài đặt thư viện

In [ ]:
# cài lib. note: cài xong phải restart runtime
!pip install pydub
!pip install pomegranate

In [ ]:
# Xác thực google để upload/download qua google drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Quản lý file, folder
import os
from shutil import copyfile, rmtree
import random

# Xử lý audio
import librosa
import librosa.display
from scipy.io import wavfile

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pomegranate # Thư viện cho mô hình xác suất
from pomegranate import *

# Tải dữ liệu và lấy xác thực Google

Dữ liệu dùng để huấn luyện và đánh giá, còn xác thực google thì dùng để upload/download mô hình trên drive.

In [ ]:
%%capture
# download wolfram
if not os.path.isfile('./dataset_1_wolfram.zip'):
  !gdown --id 115tIAitBNeJC0DwrP-ZyJ6RS3TyWN0qD
  !unzip -o dataset_1_wolfram.zip

# dowload FSDD
if not os.path.isfile('./dataset_2_FSDD.zip'):
  !gdown --id 1Ua9zlPBc0Fv4xGHSQTb7eIvUh_dqFI6P
  !unzip -o dataset_2_FSDD.zip

# download self recorded audio
!gdown --id 1lH_k1AYMVlJvodtZdD7OK2zkdPXxlW9i

In [ ]:
# Lấy xác thực google để upload/download file
auth.authenticate_user()
gauth =  GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Hàm xử lý âm thanh

In [ ]:
def scaleAddNoise(wave):
  return ((wave - wave.mean()) / wave.std()) + 0.2*np.random.normal(size=wave.shape)

def read_process_audio(full_audio_path):
  # Bắt buộc âm thanh đọc vào phải cùng sample rate
  sample_rate, wave =  wavfile.read(full_audio_path)
  if sample_rate != 16000:
    wave, sample_rate =  librosa.load(full_audio_path, sr=16000) # Hàm đọc của librosa quá chậm, scipy nhanh gấp 5-10 lần
  else:
    wave=wave/32768 # Chuẩn hóa về số thực
  wave = np.pad(wave, (2000,2000), 'constant', constant_values=(0.0,0.0))
  wave = scaleAddNoise(wave)
  return wave, sample_rate

def extract_mfcc(wave, sample_rate):
  mfccs = librosa.feature.mfcc(y=wave, sr=sample_rate, n_mfcc=n_mfcc_ceptrum+2, n_mels=40, lifter=40)[2:,:].T/800
  return mfccs

def mfcc_delta_features(mfcc, order):
  if order==0:
    return mfcc
  dmfcc = librosa.feature.delta(mfcc, order=order)
  return dmfcc

def full_mfcc_from_file(full_audio_path):
  wave, sr = read_process_audio(full_audio_path)
  mfccs = extract_mfcc(wave, sr)
  full_mfccs = mfccs
  if n_delta_features >= 1:
    mfccs_d1 = mfcc_delta_features(mfccs, 1)
    full_mfccs = np.hstack([full_mfccs,mfccs_d1])
  if n_delta_features >= 2:
    mfccs_d2 = mfcc_delta_features(mfccs, 2)
    full_mfccs = np.hstack([full_mfccs,mfccs_d2])
  return full_mfccs

# Hàm bổ trợ

In [ ]:
def buildDataSet(dir):
    # Filter out the wav audio files under the dir
    fileList = [f for f in os.listdir(dir) if os.path.splitext(f)[1] == '.wav']
    dataset = {}
    for fileName in fileList:
        tmp = fileName.split('.')[0]
        label = tmp.split('_')[1]

        # label = filename.split('_')[0]
        feature = full_mfcc_from_file(dir+fileName)
        if label not in dataset.keys():
            dataset[label] = []
            dataset[label].append(feature)
        else:
            exist_feature = dataset[label]
            exist_feature.append(feature)
            dataset[label] = exist_feature
    return dataset

def makeTrainTestDir(mainFolder, filenames, rate):
    paths = [f'{mainFolder}',
             f'{mainFolder}/train_audio',
             f'{mainFolder}/test_audio']

    for path in paths:
        try:
            os.mkdir(path)
        except:
            rmtree(path)
            os.mkdir(path)

    random.seed(1)
    random.shuffle(filenames)
    splitPoint = int(len(filenames)*rate)
    trainFilenames = filenames[:splitPoint]
    testFilenames = filenames[splitPoint:]

    for filename in trainFilenames:
        copyfile(filename[0], f"{paths[1]}/{filename[1]}")

    for filename in testFilenames:
        copyfile(filename[0], f"{paths[2]}/{filename[1]}")

# Setup thư mục chứa data

In [ ]:
def formatFilenameFSDD(dir):
    filenames = []
    count = 0
    for filename in os.listdir(dir):
        tmp = str(count) + '_' + filename.split('_')[0] + '.wav'
        filenames.append((f"{dir}/{filename}", tmp))
        count += 1
    return filenames

filenames = formatFilenameFSDD('./dataset_2_FSDD')
makeTrainTestDir('fsdd', filenames, fsdd_split)

In [ ]:
def formatFilenameWolfram(dir):
    filenames = []
    count = 0
    folders = os.listdir(dir)
    for folder in folders:
        for filename in os.listdir(f'{dir}/{folder}'):
            tmp = str(count) + '_' + folder.split('_')[0] + '.wav'
            filenames.append((f"{dir}/{folder}/{filename}", tmp))
            count += 1

    return filenames

filenames = formatFilenameWolfram('./dataset_1_wolfram')
makeTrainTestDir('wolfram', filenames, wolfram_split)

# Huấn luyện mô hình (có thể bỏ qua vì mô hình đã save trên drive)

## Hàm train mô hình

In [ ]:
def Generate_DiagGMM(full_fset, n_features, n_states=5, n_cmps=3):
  means = np.mean(full_fset, axis=0)
  stds = np.std(full_fset, axis=0)
  # initial values for all gaussian components
  np.random.seed(None)
  dist_init = np.random.random((n_states, n_cmps, n_features, 2))
  dist_init[..., 0] -= 0.5  # center means to 0.0
  #for feat_i in range(n_features):
      # random init mean in range [-std, std)
      #dist_init[..., feat_i, 0] *= 2 * stds[feat_i]
      #dist_init[..., feat_i, 0] += means[feat_i]
      # random init std in range 1std/n_components
      #dist_init[..., feat_i, 1] *= stds[feat_i] / n_cmps

  dists = tuple(
      GeneralMixtureModel(list(
          IndependentComponentsDistribution(tuple(
              NormalDistribution(*dist_init[state_i, cmp_i, feat_i, :])
              for feat_i in range(n_features)
          ))
          for cmp_i in range(n_cmps)
      ))
      if n_cmps > 1 else
      IndependentComponentsDistribution(tuple(
          NormalDistribution(*dist_init[state_i, 0, feat_i, :])
          for feat_i in range(n_features)
      ))
      for state_i in range(n_states)
  )
  return dists

In [ ]:
# Generate progressive HMM model
def left_right_GMMHMM(seed_sample, x_dim, n_states=10, n_modals=9, random=0):
  rng = np.random.RandomState(random)
  if random!=0:
    init_prob = lambda: rng.rand(1)[0]
    
  model = HiddenMarkovModel()
  states = [State( state_model, name=f"H{i}" ) for i, state_model in enumerate(Generate_DiagGMM(seed_sample, x_dim, n_states, n_modals))]
  model.add_states(states)
  model.add_transition(model.start, states[0], 1)
  for i in range(n_states-1):
    model.add_transition(states[i], states[i], 0.5)
    model.add_transition(states[i], states[i+1], 0.5)
  model.add_transition(states[n_states-1], states[n_states-1], 0.5)
  model.add_transition(states[n_states-1], model.end, 0.5)
  model.bake()
  return model

In [ ]:
def train_GMMHMM(dataset, input_dim, n_hidden_state, n_gauss_modal):
    GMMHMM_Models = dict()

    for label in dataset.keys():
        print(f"Training model detect {label}")
        model = left_right_GMMHMM(dataset[label][0], input_dim, n_hidden_state, n_gauss_modal)
        model.fit(dataset[label], verbose=True, multiple_check_input=False)  # get optimal parameters
        GMMHMM_Models[label] = model

    return GMMHMM_Models

## Hàm test mô hình

In [ ]:
# test model
def evaluateModel(testDataSet, model):
    if (len(testDataSet) == 0):
      return
    score_cnt = 0
    total = 0
    for label in testDataSet.keys():
        features = testDataSet[label]
        total += len(features)
        for sample in features:
            scoreList = {}
            for model_label in model.keys():
                score = model[model_label].log_probability(sample)
                scoreList[model_label] = score
            predict = max(scoreList, key=scoreList.get)
            if predict == label:
                score_cnt += 1
        print(f"{label}: ({score_cnt}/{total})")

    print("Final recognition rate is %.2f"%(100.0*score_cnt/total), "%")

## Đọc và tiền xử lý data

In [ ]:
# prepare data for training
master_path = 'fsdd'

trainDir = master_path + '/train_audio/'
trainDataSet_fsdd = buildDataSet(trainDir)
print("Finish prepare the training data")

# prepare data for testing
testDir = master_path + '/test_audio/'
testDataSet_fsdd = buildDataSet(testDir)
print("Finish prepare the test data")

Finish prepare the training data
Finish prepare the test data


In [ ]:
# prepare data for training
master_path = 'wolfram'

trainDir = master_path + '/train_audio/'
trainDataSet_wolfram = buildDataSet(trainDir)
print("Finish prepare the training data")

# prepare data for testing
testDir = master_path + '/test_audio/'
testDataSet_wolfram = buildDataSet(testDir)
print("Finish prepare the test data")

Finish prepare the training data
Finish prepare the test data


In [ ]:
trainDataSet = trainDataSet_fsdd
if wolfram_split > 0:
  for label in trainDataSet_fsdd.keys():
    trainDataSet[label] = trainDataSet_fsdd[label] + trainDataSet_wolfram[label]

## **Mô hình 1**
- 10 hiddent states
- Multivariate Diagonal Gauss cho emission probs

In [ ]:
%%time
# train
hmmModels = train_GMMHMM(trainDataSet, n_mfcc_features,10,1)
print("Finish training of the GMM_HMM models for digits 0-9")

Training model detect 1
[1] Improvement: 20786039.729101017	Time (s): 0.7244
[2] Improvement: 306539.71350261243	Time (s): 0.7233
[3] Improvement: 101148.51209110254	Time (s): 0.737
[4] Improvement: 44223.29904327728	Time (s): 0.7184
[5] Improvement: 10590.84284379799	Time (s): 0.7222
[6] Improvement: 6934.383041583467	Time (s): 0.7093
[7] Improvement: 6494.088202396873	Time (s): 0.7137
[8] Improvement: 7363.102380162105	Time (s): 0.7182
[9] Improvement: 15365.037946056109	Time (s): 0.7157
[10] Improvement: 12846.923371161334	Time (s): 0.7271
[11] Improvement: 6720.549340732396	Time (s): 0.7265
[12] Improvement: 3388.372663229704	Time (s): 0.7097
[13] Improvement: 1412.091207885649	Time (s): 0.7217
[14] Improvement: 680.6620311168954	Time (s): 0.7156
[15] Improvement: 494.95734082488343	Time (s): 0.7148
[16] Improvement: 1208.5574615793303	Time (s): 0.7249
[17] Improvement: 4299.5034197978675	Time (s): 0.728
[18] Improvement: 2359.8786505633034	Time (s): 0.7327
[19] Improvement: 782.98

In [ ]:
print("Evaluate on train set")
evaluateModel(trainDataSet, hmmModels)
print("Evaluate on rest of FSDD set")
evaluateModel(testDataSet_fsdd, hmmModels)
print("Evaluate on wolfram test set")
evaluateModel(testDataSet_wolfram, hmmModels)

Evaluate on train set
1: (826/1614)
3: (1839/3193)
2: (2451/4755)
7: (3437/6347)
6: (4472/7971)
8: (5546/9544)
0: (6801/11144)
9: (7844/12761)
5: (9118/14355)
4: (10408/15999)
Final recognition rate is 65.05 %
Evaluate on rest of FSDD set
0: (94/113)
3: (175/243)
9: (238/358)
7: (348/503)
4: (445/618)
5: (561/739)
1: (646/859)
2: (698/973)
6: (738/1081)
8: (849/1200)
Final recognition rate is 70.75 %
Evaluate on wolfram test set
4: (712/913)
1: (1184/1849)
6: (1794/2786)
8: (2398/3746)
0: (3141/4709)
7: (3680/5649)
5: (4411/6591)
2: (4780/7588)
3: (5374/8535)
9: (5991/9467)
Final recognition rate is 63.28 %


In [ ]:
# lưu lại model
for model_label in hmmModels.keys():
  file = drive.CreateFile({'title': f'hmm[{model_label}]_{n_mfcc_features}_10_1[{experiment_id}].json', 'parents': [{'id': '1QPUr4vwYHu3n9iH3iQmnvDUt2Dgx4V3Y'}]})
  file.SetContentString(hmmModels[model_label].to_json())
  file.Upload()

## **Mô hình 2**
- 5 hiddent states
- Multivariate Diagonal Gauss cho emission probs

In [ ]:
%%time
# train
hmmModels = train_GMMHMM(trainDataSet, n_mfcc_features,5,1)
print("Finish training of the GMM_HMM models for digits 0-9")

Training model detect 1
[1] Improvement: 4600638.992300174	Time (s): 0.4225
[2] Improvement: 113294.66526779719	Time (s): 0.43
[3] Improvement: 2778.5037671392784	Time (s): 0.526
[4] Improvement: 62301.02968573058	Time (s): 0.4136
[5] Improvement: 200837.86667790962	Time (s): 0.4314
[6] Improvement: 13786.852202899754	Time (s): 0.4195
[7] Improvement: 13289.590906445868	Time (s): 0.4253
[8] Improvement: 15163.240021197591	Time (s): 0.4583
[9] Improvement: 9002.463655491825	Time (s): 0.4245
[10] Improvement: 3526.890375703573	Time (s): 0.4265
[11] Improvement: 1863.2543101552874	Time (s): 0.4248
[12] Improvement: 1461.6332689761184	Time (s): 0.4178
[13] Improvement: 992.3231366458349	Time (s): 0.4219
[14] Improvement: 694.2600533044897	Time (s): 0.4177
[15] Improvement: 481.90232513519004	Time (s): 0.404
[16] Improvement: 425.75892256060615	Time (s): 0.4041
[17] Improvement: 280.00854836963117	Time (s): 0.4349
[18] Improvement: 127.59734848933294	Time (s): 0.4073
[19] Improvement: 40.83

In [ ]:
print("Evaluate on train set")
evaluateModel(trainDataSet, hmmModels)
print("Evaluate on rest of FSDD set")
evaluateModel(testDataSet_fsdd, hmmModels)
print("Evaluate on wolfram test set")
evaluateModel(testDataSet_wolfram, hmmModels)

Evaluate on train set
1: (674/1614)
3: (1423/3193)
2: (1938/4755)
7: (2996/6347)
6: (4165/7971)
8: (4671/9544)
0: (4682/11144)
9: (5679/12761)
5: (6424/14355)
4: (7188/15999)
Final recognition rate is 44.93 %
Evaluate on rest of FSDD set
0: (1/113)
3: (88/243)
9: (155/358)
7: (273/503)
4: (351/618)
5: (416/739)
1: (482/859)
2: (533/973)
6: (583/1081)
8: (639/1200)
Final recognition rate is 53.25 %
Evaluate on wolfram test set
4: (413/913)
1: (781/1849)
6: (1477/2786)
8: (1768/3746)
0: (1770/4709)
7: (2344/5649)
5: (2749/6591)
2: (3058/7588)
3: (3480/8535)
9: (4092/9467)
Final recognition rate is 43.22 %


In [ ]:
# lưu lại model
for model_label in hmmModels.keys():
  file = drive.CreateFile({'title': f'hmm[{model_label}]_{n_mfcc_features}_5_1[{experiment_id}].json', 'parents': [{'id': '1QPUr4vwYHu3n9iH3iQmnvDUt2Dgx4V3Y'}]})
  file.SetContentString(hmmModels[model_label].to_json())
  file.Upload()

## **Mô hình 3**
- 3 hiddent states
- Multivariate Diagonal Gauss cho emission probs

In [ ]:
%%time
# train
hmmModels = train_GMMHMM(trainDataSet, n_mfcc_features,3,1)
print("Finish training of the GMM_HMM models for digits 0-9")

Training model detect 1
[1] Improvement: 3958422.188400025	Time (s): 0.269
[2] Improvement: 62008.39168069791	Time (s): 0.2827
[3] Improvement: 3554.658271588385	Time (s): 0.2692
[4] Improvement: 2909.7981928964145	Time (s): 0.2777
[5] Improvement: 1843.2728583044372	Time (s): 0.2758
[6] Improvement: 837.9768205475993	Time (s): 0.286
[7] Improvement: 447.85777894128114	Time (s): 0.275
[8] Improvement: 294.88940485566854	Time (s): 0.2887
[9] Improvement: 224.52443162398413	Time (s): 0.2785
[10] Improvement: 188.35678756330162	Time (s): 0.2873
[11] Improvement: 113.00699347490445	Time (s): 0.2874
[12] Improvement: 97.8375198864378	Time (s): 0.3905
[13] Improvement: 91.40229683648795	Time (s): 0.2959
[14] Improvement: 51.509555120486766	Time (s): 0.2735
[15] Improvement: 31.709827408194542	Time (s): 0.2753
[16] Improvement: 21.255953990854323	Time (s): 0.2837
[17] Improvement: 25.941990664228797	Time (s): 0.2958
[18] Improvement: 8.866574324667454	Time (s): 0.2727
[19] Improvement: 27.227

In [ ]:
print("Evaluate on train set")
evaluateModel(trainDataSet, hmmModels)
print("Evaluate on rest of FSDD set")
evaluateModel(testDataSet_fsdd, hmmModels)
print("Evaluate on wolfram test set")
evaluateModel(testDataSet_wolfram, hmmModels)

Evaluate on train set
1: (85/1614)
3: (134/3193)
2: (137/4755)
7: (827/6347)
6: (1425/7971)
8: (2291/9544)
0: (3097/11144)
9: (4200/12761)
5: (4343/14355)
4: (5483/15999)
Final recognition rate is 34.27 %
Evaluate on rest of FSDD set
0: (46/113)
3: (47/243)
9: (111/358)
7: (210/503)
4: (308/618)
5: (316/739)
1: (317/859)
2: (317/973)
6: (343/1081)
8: (438/1200)
Final recognition rate is 36.50 %
Evaluate on wolfram test set
4: (628/913)
1: (680/1849)
6: (1047/2786)
8: (1520/3746)
0: (2026/4709)
7: (2443/5649)
5: (2526/6591)
2: (2530/7588)
3: (2571/8535)
9: (3249/9467)
Final recognition rate is 34.32 %


In [ ]:
# lưu lại model
for model_label in hmmModels.keys():
  file = drive.CreateFile({'title': f'hmm[{model_label}]_{n_mfcc_features}_3_1[{experiment_id}].json', 'parents': [{'id': '1QPUr4vwYHu3n9iH3iQmnvDUt2Dgx4V3Y'}]})
  file.SetContentString(hmmModels[model_label].to_json())
  file.Upload()